In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/Projet SatMap/Implementation

/content/drive/My Drive/Projet SatMap/Implementation


In [3]:
!pip install tifffile

     |████████████████████████████████| 133kB 2.8MB/s 
     |████████████████████████████████| 18.1MB 188kB/s 


In [4]:
# Test du fichier "treatment.py" pour la partie "X_train" et "Y_train"

import shapely.affinity
import tifffile as tiff
import matplotlib.pyplot
import csv
import sys

%matplotlib inline

import bin.data_opening as op
import bin.treatment as tr

csv.field_size_limit(sys.maxsize)

# Liste des images annotees
LIST_IMG_ID = ('6010_1_2', '6010_4_2', '6010_4_4', '6040_1_0', '6040_1_3', '6040_2_2', 
         '6040_4_4', '6060_2_3', '6070_2_3', '6090_2_0', '6100_1_3', '6100_2_2',
         '6100_2_3', '6110_1_2', '6110_3_1', '6110_4_0', '6120_2_0', '6120_2_2',
         '6140_1_2', '6140_3_1', '6150_2_3', '6160_2_1', '6170_0_4', '6170_2_4',
         '6170_4_1')
# ID des classes respectives : batiments, routes, arbres, rivieres, lacs 
LIST_POLY_TYPE = ('1', '3', '5', '7', '8')
#IMG_ID = LIST_IMG_ID[2]
POLY_TYPE = LIST_POLY_TYPE[2]

LIST_MASK = []
LIST_IMG  = []

for IMG_ID in LIST_IMG_ID:

    # Ouverture des fichiers
    data_opening = op.DataOpening()

    x_max, y_min = data_opening.get_size(IMG_ID)
    polygons = data_opening.get_polygons(IMG_ID, POLY_TYPE)
    img, img_size = data_opening.get_tiff(IMG_ID)

    # Traitment des fichiers
    treatment = tr.Treatment()

    x_scaler, y_scaler = treatment.get_scalers(img_size, x_max, y_min)

    list_polygons = shapely.affinity.scale(polygons, xfact=x_scaler, yfact=y_scaler, origin=(0, 0, 0))

    mask = treatment.get_mask_polygons(img_size, list_polygons)

    img, mask = treatment.set_size(img, mask)

    # Sauvegarder les donnees
    LIST_IMG.append(img)
    LIST_MASK.append(mask)

print("taille LIST_MASK :", len(LIST_MASK))
print("taille LIST_IMG :", len(LIST_IMG))

X_train, Y_train = treatment.get_data_train(LIST_IMG, LIST_MASK)

print("X_train shape :", X_train.shape)
print("Y_train shape :", Y_train.shape)


taille LIST_MASK : 25
taille LIST_IMG : 25
X_train shape : (11165610, 75)
Y_train shape : (11165610, 75)
